<a href="https://colab.research.google.com/github/gmoriki/C4RA-Python-Tutorials/blob/main/0616/0616_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第6回C4RA勉強会
## DataFrameのデータ統計と集計
---

## ★ 勉強会の基本方針

* 5月12日(金)から6月30日(金)までの計8回を予定しています。
* 目標は**pandas.DataFrame形式の基本的な操作をマスターすること**です。

* 環境はGoogle Colaboratory(通称Colab)を使用します。Googleアカウントをご用意ください。
  * 各々のローカル環境でも実行可能です。
* 一部、東京大学「Pythonプログラミング入門」の教材を共有しながら勉強会を進めます。
  * 勉強会では**東大教材**と呼びます
  * URL：https://utokyo-ipp.github.io/index.html

* Colab（Jupyter Notebook）の詳しい使い方について、勉強会の中で深入りはしません
  * Colab立ち上げのURL：https://colab.research.google.com/
  * 便利なショートカット：https://blog.kikagaku.co.jp/google-colaboratory-shortcut

* Pythonに関する不明点があれば森木(Pumble,Mail,Twitter等)までご連絡ください
* 勉強会の内容でなくてもOKです（Colabの使い方や環境構築なども）
---

## ■ はじめに(データの型)

In [ ]:
import pandas as pd
import numpy as np

# データフレーム作成
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35],
        'City': ['New York', 'San Francisco', 'Los Angeles']}
df = pd.DataFrame(data)

In [ ]:
df.dtypes

In [ ]:
df['new_col'] = [1,2,'aiueo']

In [ ]:
df

In [ ]:
df.dtypes

以下の科研費データを使用します。

In [ ]:
## ファイルの取得
!wget -P . https://raw.githubusercontent.com/gmoriki/C4RA-Python-Tutorials/master/0609/kaken.nii.ac.jp_2023-06-07_10-57-32.csv

In [ ]:
df_kaken = pd.read_csv('./kaken.nii.ac.jp_2023-06-07_10-57-32.csv')
df_kaken.head(5)

tail = しっぽ

In [ ]:
df_kaken.tail(3)

データを扱うために前処理を行います。

In [ ]:
# 列の選択
columns_list = ['研究課題名','審査区分','研究種目','研究機関','総配分額','総配分額 (直接経費)','総配分額 (間接経費)']
df_kaken = df_kaken[columns_list].copy()

# 百万円単位に変換する
# いろいろな百万円の表現を使っています

df_kaken['総配分額[百万]'] = df_kaken['総配分額'] / 1000000
df_kaken['総配分額 (直接経費)[百万]'] = df_kaken['総配分額 (直接経費)'] / 1e6
df_kaken['総配分額 (間接経費)[百万]'] = df_kaken['総配分額 (間接経費)'] / 1_000_000

df_kaken

In [ ]:
# 列の削除
df_kaken = df_kaken.drop(['総配分額','総配分額 (直接経費)','総配分額 (間接経費)'], axis=1) # 列方向に削除したいのでaxis=1

# 元の列名をもう一度設定
df_kaken.columns = columns_list
df_kaken

(参考)最初からこうしておけばよりスムーズです。

In [ ]:
# (参考)最初からこうしておけば良かったコード
# df_kaken['総配分額'] = df_kaken['総配分額'] / 1000000
# df_kaken['総配分額 (直接経費)'] = df_kaken['総配分額 (直接経費)'] / 1e6
# df_kaken['総配分額 (間接経費)'] = df_kaken['総配分額 (間接経費)'] / 1_000_000

研究課題名はダミーに置き換えます。

In [ ]:
df_kaken['研究課題名'] = ["課題No." + str(idx) for idx in df_kaken.index]
df_kaken

In [ ]:
df_kaken_origin = df_kaken.copy()

(参考)上記コードは"リスト内包表記"という便利な書き方です。

内容は以下のコードと同じです。

In [ ]:
# new_values = [] # 空のリストを作成

# for idx in df_kaken.index:
#     new_value = "課題No." + str(idx) # 新しい値を作成
#     new_values.append(new_value) # リストに新しい値を追加

# df_kaken['研究課題名'] = new_values # 新しい値のリストをDataFrameの列に設定

## ■ データの統計

PandasのDataFrameには、データの基本的な統計を取得するための便利なメソッドが多数用意されています。

* mean(): 平均値を計算します。各列の数値データをすべて足し合わせて、データの個数で割った値です。
* median(): 中央値（全データを数値の大きさで並べたときにちょうど中央にくる値）を計算します。データの個数が偶数の場合は、中央の2つの値の平均が中央値となります。
* max(): 最大値を計算します。各列の数値データの中で最も大きな値を返します。
* min(): 最小値を計算します。各列の数値データの中で最も小さな値を返します。
* std(): 標準偏差（データのばらつき具合を数値化したもの）を計算します。データの平均値からどれだけデータが散らばっているかを示します。

これらのメソッドは、数値データに対してのみ機能します。文字列や日付などの非数値データが含まれる列に対しては、これらのメソッドは無視されます。

In [ ]:
df_kaken['総配分額'].mean()

In [ ]:
# 各統計量を計算
age_mean = df_kaken['総配分額'].mean()
age_median = df_kaken['総配分額'].median()
age_max = df_kaken['総配分額'].max()
age_min = df_kaken['総配分額'].min()
age_std = df_kaken['総配分額'].std()
age_count = df_kaken['総配分額'].count()
age_q1 = df_kaken['総配分額'].quantile(0.25)
age_q3 = df_kaken['総配分額'].quantile(0.75)

# 結果を表示
print('Count: ', age_count)
print('Mean: ', age_mean)
print('Standard Deviation: ', age_std)
print('Minimum: ', age_min)
print('25th Percentile: ', age_q1)
print('Median: ', age_median)
print('75th Percentile: ', age_q3)
print('Maximum: ', age_max)

In [ ]:
# 各統計量を計算
age_mean = df_kaken['総配分額'].mean()
age_median = df_kaken['総配分額'].median()
age_max = df_kaken['総配分額'].max()
age_min = df_kaken['総配分額'].min()
age_std = df_kaken['総配分額'].std()
age_count = df_kaken['総配分額'].count()
age_q1 = df_kaken['総配分額'].quantile(0.25)
age_q3 = df_kaken['総配分額'].quantile(0.75)

# 結果を表示
print(f"Count: {age_count}\nMean: {age_mean}\nStandard Deviation: {age_std}\nMinimum: {age_min}\n25th Percentile: {age_q1}\nMedian: {age_median}\n75th Percentile: {age_q3}\nMaximum: {age_max}")

以上のコードは一つのメソッドで実現できます。

In [ ]:
df_kaken['総配分額'].describe()

## ■ データの集計

次に、データの集計について学びましょう。

PandasのDataFrameには、データの集計を行うための便利なメソッドが多数用意されています。

* sum(): 各列の合計値を計算します。数値データをすべて足し合わせた値を返します。
* count(): 各列の個数（非欠損値の数）を計算します。欠損値（NaN）を除いたデータの個数を返します。
* groupby(): 特定の列を基準にして、データをグループ化します。グループ化した後に、各グループに対して平均（mean()）、合計（sum()）などの集計操作を行うことができます。

これらのメソッドを組み合わせることで、さまざまなデータ集計を行うことが可能になります。

In [ ]:
df_kaken = df_kaken_origin.copy()

In [ ]:
columns_groupby_category = ['研究種目','総配分額','総配分額 (直接経費)','総配分額 (間接経費)']
df_kaken = df_kaken[columns_groupby_category]

In [ ]:
df_kaken.sum()

In [ ]:
df_kaken.count()

groupbyが本番です。

参考URL：https://note.nkmk.me/python-pandas-groupby-statistics/

In [ ]:
df_kaken.groupby('研究種目')

In [ ]:
grouped_category = df_kaken.groupby('研究種目')
grouped_category.size()

In [ ]:
grouped_category.sum()

In [ ]:
grouped_category.min()

In [ ]:
grouped_category.agg(['sum','min'])

In [ ]:
grouped_category.agg(['max','min'])

In [ ]:
def my_func(x):
    return max(x) - min(x)

grouped_category.agg(my_func)

上記と同じ処理をラムダ式で表現。

In [ ]:
grouped_category.agg(lambda x: max(x) - min(x))

In [ ]:
grouped_category.agg({'総配分額':['sum','mean'],'総配分額 (直接経費)':'mean','総配分額 (間接経費)':'max'})

In [ ]:
new_order = ['基盤研究(C)', '基盤研究(B)', '基盤研究(A)', '基盤研究(S)']
grouped_category.agg({'総配分額':'sum','総配分額 (直接経費)':'mean','総配分額 (間接経費)':'max'}).loc[new_order]

In [ ]:
columns_groupby_category = ['研究種目','総配分額','総配分額 (直接経費)','総配分額 (間接経費)']
df_kaken = df_kaken[columns_groupby_category]

複数のカラムをグループ化してみましょう。

In [ ]:
df_kaken = df_kaken_origin.copy()
columns_groupby_category_inst = ['研究種目','研究機関','総配分額','総配分額 (直接経費)','総配分額 (間接経費)']
df_kaken = df_kaken[columns_groupby_category_inst].copy()

In [ ]:
# df_kaken = df_kaken[df_kaken['研究機関'].str.contains('東京大学') | df_kaken['研究機関'].str.contains('京都大学')]
df_kaken_2top = df_kaken[df_kaken['研究機関'].isin(['東京大学', '京都大学'])]
df_kaken_2top

In [ ]:
grouped_category_2top = df_kaken_2top.groupby(['研究種目','研究機関'])
grouped_category_2top.size()

In [ ]:
grouped_category_2top.sum()

In [ ]:
grouped_category_2top.agg({'総配分額':['sum'],'総配分額 (直接経費)':'mean','総配分額 (間接経費)':'max'})

In [ ]:
df_2top_table = grouped_category_2top.agg({'総配分額':['sum','mean'],'総配分額 (直接経費)':'mean','総配分額 (間接経費)':'max'})

In [ ]:
df_2top_table.reset_index()

In [ ]:
level0 = df_2top_table.columns.get_level_values(0)
level1 = df_2top_table.columns.get_level_values(1)

df_2top_table.columns = level0 + '_' + level1

df_2top_table

In [ ]:
df_2top_table.reset_index()

## ■ オマケ：パイプラインについて

In [ ]:
df_kaken = pd.read_csv('./kaken.nii.ac.jp_2023-06-07_10-57-32.csv')

In [ ]:
# copy
df_kaken_1 = df_kaken.copy()
# こうやって書くと大変

# 欠損値を削除
df_kaken_1 = df_kaken_1.dropna(axis=1, how='any')

# 必要な列だけを選択
df_kaken_1 = df_kaken_1[['研究課題名','研究種目','総配分額']]

# 条件に合う行を選択
mask = (df_kaken_1['総配分額'] > df_kaken_1['総配分額'].mode().values[0]) & (df_kaken_1['研究種目'].str.contains('C'))
df_kaken_1 = df_kaken_1[mask]

# '総配分額'でソート
df_kaken_1 = df_kaken_1.sort_values(by='総配分額')

# インデックスをリセット
df_kaken_1 = df_kaken_1.reset_index(drop=True)
df_kaken_1

In [ ]:
# copy
df_kaken_2 = df_kaken.copy()
# 複数の処理を続けて書くことができます(パイプライン)
df_kaken_2 = df_kaken_2.dropna(axis=1, how='any')[['研究課題名','研究種目','総配分額']][(df_kaken_2['総配分額'] > df_kaken_2['総配分額'].mode().values[0]) & (df_kaken_2['研究種目'].str.contains('C'))].sort_values(by='総配分額').reset_index(drop=True)
df_kaken_2

In [ ]:
# copy
df_kaken_3 = df_kaken.copy()

# 見やすい例1
df_kaken_3 = (df_kaken_3.dropna(axis=1, how='any') \
            [['研究課題名','研究種目','総配分額']] \
            [(df_kaken_3['総配分額'] > df_kaken_3['総配分額'].mode().values[0]) & (df_kaken_3['研究種目'].str.contains('C'))] \
            .sort_values(by='総配分額') \
            .reset_index(drop=True)
            ) 
df_kaken_3

In [ ]:
# copy
df_kaken_4 = df_kaken.copy()

# 見やすい例2
def drop_na_cols(df):
    return df.dropna(axis=1, how='any')

def filter_rows(df):
    return df[(df['総配分額'] > df['総配分額'].mode()[0]) & (df['研究種目'].str.contains('C'))]

def select_columns(df):
    return df[['研究課題名','研究種目','総配分額']]

def sort_values(df):
    return df.sort_values(by='総配分額')

def reset_idx(df):
    return df.reset_index(drop=True)

df_kaken_4 = (df_kaken_4
            .pipe(drop_na_cols)
            .pipe(filter_rows)
            .pipe(select_columns)
            .pipe(sort_values)
            .pipe(reset_idx)
           )
df_kaken_4

In [ ]:
# copy
df_kaken_5 = df_kaken.copy()

# 見やすい例3
def process_dataframe(df):
    df = df.dropna(axis=1, how='any')
    df = df[(df['総配分額'] > df['総配分額'].mode()[0]) & (df['研究種目'].str.contains('C'))]
    df = df[['研究課題名','研究種目','総配分額']]
    df = df.sort_values(by='総配分額')
    df = df.reset_index(drop=True)
    return df

df_kaken_5 = df_kaken_5.pipe(process_dataframe)
df_kaken_5